In [1]:
import cv2
import time
import imutils 
from collections import deque
import math
import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob
import imageio

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


In [2]:
class EuclideanDistTracker:
    def __init__(self):
        # Storing the positions of center of the objects
        self.center_points = {}
        # Count of ID of boundng boxes
        # each time new object will be captured the id will be increassed by 1
        self.id_count = 0
    
    def update(self, objects_rect):
        
        objects_bbs_ids = []
        
        # Calculating the center of objects
        for rect in objects_rect:
            x, y, w, h = rect
            center_x = (x + x + w) // 2
            center_y = (y + y + h) // 2
            
            # Find if object is already detected or not
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(center_x - pt[0], center_y - pt[1])
                if dist < 25:
                    self.center_points[id] = (center_x, center_y)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])     
                    same_object_detected = True
                    break
            
            # Assign the ID to the detected object
            if same_object_detected is False:
                self.center_points[self.id_count] = (center_x, center_y)                      
                objects_bbs_ids.append([x, y, w, h, self.id_count])       
                self.id_count += 1
        
        # Cleaning the dictionary ids that are not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            var,var,var,var, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center
       
        # Updating the dictionary with IDs that is not used
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [3]:
def getContours(img,imgContour):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        #Debugging statements
        # if area > 1:
        #     print("Area of contour is: {}".format(area))
        
        areaMin = 3
        areaMax = 35
        if area > areaMin and area < areaMax:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x, y), (x + w, y + h), (0, 255, 0), 5)
            print(x,y,w,h)


In [5]:
#videos to choose from
NeedleViz_path1 = 'Data/edited data/102622_Water.mp4'
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4'
NeedleViz_oilAndLatex = 'Data/edited data/oil and latex/capture_5_2022-11-12T16-56-03.mp4'
NeedleViz_gelAndLatex = 'Data/edited data/ultrasound gel and latex/capture_4_2022-11-12T17-33-19.mp4'

#control playback speed
frame_rate = 30

# vc = cv2.VideoCapture(0) #opens camera
vc = cv2.VideoCapture(NeedleViz_path2)

fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40) #pretty good: (100,200)
tracker = EuclideanDistTracker() #creating tracker object

frameWidth = 440
frameHeight = 440
vc.set(3, frameWidth)
vc.set(4, frameHeight)

size = (frameWidth, frameHeight)

#Preparing to create output videos
image_lst = []


if (vc.isOpened()== False): 
  print("Error opening video  file")

while(vc.isOpened()):
    rval, frame = vc.read()
    
    if rval == True:


        #Initial Frame preprocessing
        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        resized_gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_RGB2GRAY)


        #Achieving desired region of interest within Raw Frame
        ROI_frame = resized_gray_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = np.zeros_like(resized_gray_frame)


        x = 94 #initial column number
        y = 166 #initial row number
        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 

        imgContour2 = resized_frame.copy()


        #Object Detection
        fgmask = fgbg.apply(blank_img)
        _, fgmask = cv2.threshold(fgmask, 254, 255, cv2.THRESH_BINARY) #want only white pixels
        
        contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        detections = []
        for cnt in contours:
          area = cv2.contourArea(cnt)
      
          #Debugging statements
          # if area > 1:
          #     print("Area of contour is: {}".format(area))
          
          areaMin = 3
          areaMax = 35
          if area > areaMin and area < areaMax:
              # cv2.drawContours(imgContour2, cnt, -1, (255, 0, 255), 7)
              peri = cv2.arcLength(cnt, True)
              approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
              x, y, w, h = cv2.boundingRect(approx)
              cv2.rectangle(imgContour2, (x, y), (x + w, y + h), (0, 255, 0), 5)
              # print(x,y,w,h)
              detections.append([x,y,w,h])
        
        
        #Object Tracking
        boxes_ids = tracker.update(detections)
        for box_id in boxes_ids:
          x,y,w,h,id = box_id
          cv2.putText(imgContour2, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
          cv2.rectangle(imgContour2, (x, y), (x + w, y + h), (0, 255, 0), 3)

        

        ###########################################################################################

        # Debugging Statements
        cv2.imshow('normal frame', resized_gray_frame)
        cv2.imshow('FGmask', fgmask)
        # cv2.imshow('Basic Filters', imgBlur)
        # cv2.imshow('Canny', imgCanny_initial)
        # cv2.imshow('FGmaskV2', fgmaskV2)
        # cv2.imshow('FGmaskV2_withfilters', maskV2)
        # cv2.imshow('Contour', imgContour)
        cv2.imshow('Contour2', imgContour2)
        # cv2.imshow('trajectory', trajectory_frame)
        # print("raw image shape = {}".format(resized_frame.shape))
        # print("segmented image shape = {}".format(fgmaskV2_color.shape))
        # print("raw image shape (reverted)= {}".format(resized_frame_revert.shape))
        
        #Saving comparison frames as gif 
        resized_frame = cv2.cvtColor(resized_gray_frame, cv2.COLOR_GRAY2BGR)
        overlay = cv2.cvtColor(imgContour2, cv2.COLOR_RGB2BGR)
        stack = np.hstack((resized_frame, overlay))
        # cv2.imshow("stacked", stack)
        image_lst.append(stack)


        # Press Q on keyboard to  exit
        if cv2.waitKey(frame_rate) & 0xFF == ord('q'): #original waitkey is 25
            break
    
    #Break out of loop if video is done
    else:
        break  

vc.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

{0: (177, 341), 1: (230, 294)}
{0: (177, 341), 1: (239, 291)}
{0: (177, 341), 1: (216, 290)}
{0: (177, 341), 1: (199, 289)}
{0: (177, 341), 1: (194, 285)}
{0: (177, 341), 1: (194, 285), 2: (219, 259), 3: (249, 184)}
{4: (229, 247)}
{4: (216, 243)}
{4: (225, 244)}
{4: (222, 239)}
{6: (208, 288)}
{6: (191, 284)}
{6: (198, 284), 7: (236, 284)}
{6: (185, 284), 7: (236, 284)}
{6: (197, 285)}
{8: (221, 248)}
{8: (225, 252), 9: (229, 286), 10: (191, 283)}
{9: (235, 289), 10: (191, 283), 8: (225, 252)}
{9: (235, 289), 10: (184, 286), 8: (225, 252)}
{9: (224, 286), 10: (184, 286), 8: (225, 252)}
{9: (224, 286), 10: (190, 283), 8: (225, 252)}
{9: (224, 286), 10: (190, 283), 8: (221, 252)}
{9: (238, 288), 10: (190, 283), 8: (221, 252)}
{9: (220, 287), 10: (190, 283), 8: (221, 252)}
{9: (220, 287), 10: (184, 286), 8: (221, 252)}
{9: (242, 283), 10: (184, 286), 8: (221, 252)}
{9: (242, 283), 10: (187, 281), 8: (221, 252)}
{9: (242, 283), 10: (187, 281), 8: (220, 253)}
{9: (222, 286), 10: (187, 281)

In [7]:
imageio.mimsave('Outputs/V2_objectTracking_video_10fps.gif', image_lst, fps=10)

In [ ]:
#Reference (video): https://www.youtube.com/watch?v=O3b8lVF93jU&ab_channel=Pysource 
#reference: https://pysource.com/2021/01/28/object-tracking-with-opencv-and-python/
#reference (tracker class): https://www.analyticsvidhya.com/blog/2022/04/building-vehicle-counter-system-using-opencv/ 